In [2]:
## Get some random module
!pip install nbastats
import nbastats.nbastats as nbastats
!pip install py-goldsberry
import goldsberry

In [8]:
## Import a bunch of modules that may be helpful
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
import time
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
import requests

In [326]:
## Saving what we merged, checking the shape to see how many shots we didn't have
joined_shots.to_csv("joined_shots_2014.csv")
print joined_shots.shape,all_shots.shape,all_chart.shape

(200532, 40) (203590, 21) (205550, 22)


In [9]:
## function for getting the shot log of a player given his ID and the year
def shot_log(year,player_id):
    api_param = {'DateFrom' : '',
                    'DateTo' : '',
                    'GameSegment': '',
                    'LastNGames': 0,
                    'LeagueID': "00",
                    'Location': '',
                    'Month': 0,
                    'OpponentTeamID': 0,
                    'Outcome': '',
                    'Period': 0,
                    'PlayerID': player_id,
                    'Season': year,
                    'SeasonSegment': '',
                    'SeasonType': "Regular Season",
                    'TeamID': 0,
                    'VsConference':'',
                    'VsDivision': ''
                               }
    pull=requests.get("http://stats.nba.com/stats/playerdashptshotlog?",params=api_param)
    headers = pull.json()['resultSets'][0]['headers']
    values = pull.json()['resultSets'][0]['rowSet']
    huh=pd.DataFrame([dict(zip(headers, value)) for value in values])
    return huh


In [366]:
# get players in season a
def players(a):
    url = "http://stats.nba.com/stats/commonallplayers?"
    api_param = {'IsOnlyCurrentSeason':"0",
                          'LeagueID': '00',
                          'Season': '2015-16'}
    pull = requests.get(url, params=api_param)
    _headers = pull.json()['resultSets'][0]['headers']
    _values = pull.json()['resultSets'][0]['rowSet']
    season=pd.DataFrame([dict(zip(_headers, value)) for value in _values])
    season = season.query("FROM_YEAR <= @a and TO_YEAR >= @a")
    return season

In [ ]:
players2014=players('2014')
players2013=players('2013')

In [293]:
#Get shot chart and shot log for all players in 2014
#Don't run, already been saved
all_shots= pd.DataFrame()
all_chart = pd.DataFrame()
for index, row in players2014.iterrows():
    chart = nbastats.ShotChart(row["PERSON_ID"],season="2014-15")
    all_chart=all_chart.append(chart.shotchart())
    shots = shot_log("2014-15",row["PERSON_ID"])
    shots['PLAYER_ID']=row["PERSON_ID"]
    all_shots = all_shots.append(shots)
all_shots.to_csv("all_shots_2014.csv")
all_chart.to_csv("all_chart_2014.csv")

In [295]:
#Get shot chart and shot log for all players in 2013
#Don't run, already been saved
all_shots= pd.DataFrame()
all_chart = pd.DataFrame()
for index, row in season2013.iterrows():
    shots = shot_log("2013-14",row["PERSON_ID"])
    shots['PLAYER_ID']=row["PERSON_ID"]
    all_shots = all_shots.append(shots)
    chart = nbastats.ShotChart(row["PERSON_ID"],season="2013-14")
    all_chart=all_chart.append(chart.shotchart())
all_shots.to_csv("all_shots_2013.csv")
all_chart.to_csv("all_chart_2013.csv")

In [ ]:
## Merging shot chart data with player tracking for 2014
## DO NOT RUN... takes a while
all_shots = pd.read_csv("all_shots_2014.csv")
all_chart = pd.read_csv("all_chart_2014.csv")
players=set(all_shots['PLAYER_ID'])
joined_shots=pd.DataFrame()
joined_shots_1=pd.DataFrame()
for player in players:
    shots = all_shots.loc[all_shots.PLAYER_ID==player].reset_index(drop=True)
    chart = all_chart.loc[all_chart.PLAYER_ID==player].reset_index(drop=True)
    split_time = shots.GAME_CLOCK.str.split(':')
    col_names=[u'CLOSEST_DEFENDER', u'CLOSEST_DEFENDER_PLAYER_ID', u'CLOSE_DEF_DIST', 
               u'DRIBBLES', u'FGM', u'FINAL_MARGIN', u'GAME_CLOCK', u'GAME_ID', u'LOCATION', 
               u'MATCHUP', u'PERIOD', u'PTS', u'PTS_TYPE', u'SHOT_CLOCK', u'SHOT_DIST', 
               u'SHOT_NUMBER', u'SHOT_RESULT', u'TOUCH_TIME', u'W',u'GRID_TYPE',u'GAME_EVENT_ID',
               u'PLAYER_ID', u'PLAYER_NAME', u'TEAM_ID',u'MINUTES_REMAINING', u'SECONDS_REMAINING', 
               u'EVENT_TYPE', u'ACTION_TYPE', u'SHOT_TYPE', u'SHOT_ZONE_BASIC', 
               u'SHOT_ZONE_AREA', u'SHOT_ZONE_RANGE', u'SHOT_DISTANCE', u'LOC_X', u'LOC_Y', 
               u'SHOT_ATTEMPTED_FLAG', u'SHOT_MADE_FLAG']
    time = pd.DataFrame(shots.GAME_CLOCK.str.split(':').tolist(), columns="MIN SECONDS".split())
    shots['MIN'] = time['MIN']
    shots['SECONDS'] = time['SECONDS']
    shots['TIME']=60*shots.MIN.astype(int)+shots.SECONDS.astype(int)
    chart['TIME']=60*chart.MINUTES_REMAINING+chart.SECONDS_REMAINING
    games = set(chart["GAME_ID"])
    for game in games:
        game_shots_chart=chart.loc[chart.GAME_ID==game].reset_index(drop=True)
        game_shots_log = shots.loc[shots.GAME_ID==game].reset_index(drop=True)
        if game_shots_chart.shape[0]==game_shots_log.shape[0]:
            huh = pd.concat([game_shots_chart,game_shots_log],axis=1)
            huh=huh[col_names]
            joined_shots=joined_shots.append(huh)
        else:
            quarters=set(game_shots_chart['PERIOD'])
            for quarter in quarters:
                same_period=game_shots_log.loc[(game_shots_log.PERIOD==row['PERIOD'])].reset_index(drop=True)
                same_period_chart=game_shots_chart.loc[(game_shots_chart.PERIOD==row['PERIOD'])].reset_index(drop=True)
                if (not same_period.empty) and (not same_period_chart.empty):
                    if same_period_chart.shape[0]==same_period.shape[0]:
                        huh = pd.concat([same_period_chart,same_period],axis=1)
                        huh=huh[col_names]
                        joined_shots=joined_shots.append(huh)
                    else:
                        for index,row in same_period_chart.iterrows():
                            same_period['DIF']=abs(same_period.TIME-row.TIME)
                            df=same_period.sort(['DIF'],ascending=True)
                            same_time=same_period.loc[[0]]
                            if same_time.DIF[0]<5:
                                print "hi"
                                a=same_period_chart.loc[[index]].reset_index(drop=True)
                                huh=pd.concat([a,same_time],axis=1)
                                huh=huh[col_names]
                                joined_shots_1=joined_shots_1.append(huh,ignore_index=True)
    print player
joined_shots

In [ ]:
## Saving what we merged, checking the shape to see how many shots we didn't have
joined_shots.to_csv("joined_shots_2014.csv")
print joined_shots.shape,all_shots.shape,all_chart.shape

In [ ]:
##NEED TO DO THE SAME FOR 2013

In [345]:
joined_shots=pd.read_csv("joined_shots_2014.csv")
max(joined_shots.LOC_Y)
plt.scatter(joined_shots.LOC_X,joined_shots.LOC_Y)
plt.show()

In [350]:
huh=joined_shots[joined_shots.PTS_TYPE==2]
plt.scatter(huh.LOC_X,huh.LOC_Y)
plt.show()